In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image, ImageDraw
from matplotlib.lines import Line2D
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

In [7]:
import pandas as pd
import plotly.graph_objects as go

def create_rankings(df, years):
    rank_columns = ["rank_{}".format(i) for i in range(len(years))]
    for i, year in enumerate(years):
        df[rank_columns[i]] = df.groupby('year')['new_rating'].rank(ascending=False)
        
    return df, rank_columns

def add_text(fig, beer_name, x, y, text, font_size=12):
    fig.add_trace(go.Scatter(x=[x], y=[y], text=[text], mode="text", showlegend=False,
                             textfont=dict(size=font_size), hoverinfo='text+x+y'))

def add_pie_and_flag(fig, x, y, name, ratio):
    fig.add_trace(go.Pie(labels=[name], values=[ratio], hole=0.4,
                         marker=dict(colors=['rgba(255, 255, 255, 0)']),
                         domain=dict(x=[x-0.15, x+0.15], y=[y-0.15, y+0.15]),
                         hoverinfo='label+percent+name', showlegend=False))

years = ["2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017"]
df = pd.read_csv("./../../Project/top_10_beers_by_year.csv", index_col=None)
df, rank_columns = create_rankings(df, years)

fig = go.Figure()

for i, row in df.iterrows():
    print(row)
    for j, rc in enumerate(rank_columns):
        add_pie_and_flag(fig, j, -row[rc], name=row.beer_name, ratio=row[row[rc]] / row[rank_columns].max())

    add_text(fig, row.beer_name, len(years) - 0.5, -(i + 1), f"{row.beer_name} ({row[rank_columns[-1]]:.1f})")

fig.update_layout(
    title_text="Top 10 Beers Each Year",
    xaxis=dict(visible=False),
    yaxis=dict(visible=False),
    autosize=False,
    width=800,
    height=600,
)

fig.show()


Unnamed: 0                           1
beer_name     Weihenstephaner Original
year                            2000.0
new_rating                    1.272458
rank_0                             1.0
rank_1                             1.0
rank_2                             1.0
rank_3                             1.0
rank_4                             1.0
rank_5                             1.0
rank_6                             1.0
rank_7                             1.0
rank_8                             1.0
rank_9                             1.0
rank_10                            1.0
rank_11                            1.0
rank_12                            1.0
rank_13                            1.0
rank_14                            1.0
rank_15                            1.0
rank_16                            1.0
rank_17                            1.0
Name: 0, dtype: object


KeyError: 1.0

In [37]:
import pandas as pd
import plotly.graph_objects as go

years = ["2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017"]
df = pd.read_csv("./../../Project/top_10_beers_by_year.csv", index_col=None)
df, rank_columns = create_rankings(df, years)
# Supposons que df est votre DataFrame contenant beer_name, year, new_rating et ranks
years = sorted(df['year'].unique())
rank_columns = [f'rank_{i}' for i in range(10)]

# Remplir les rangs manquants en répétant le dernier classement disponible
df_filled = df.copy()
for year in years:
    last_ranking = None
    for rank_column in rank_columns:
        mask = (df['year'] == year) & df_filled[rank_column].isna()
        df_filled.loc[mask, rank_column] = last_ranking
        last_ranking = df_filled[mask][rank_column].max()

fig = go.Figure()

# Ajouter un scatter pour chaque année entière
scatter_data = []
for i, year in enumerate(years):
    if year.is_integer():  # Exclure les années avec des décimales
        top_10 = df_filled[df_filled['year'] == year].nlargest(10, 'new_rating')
        scatter_data.append(go.Scatter(
            x=[year] * 10,
            y=list(range(10, 0, -1)),  # Inverser l'axe y
            mode='markers+text',
            marker=dict(size=10),
            text=top_10['beer_name'],
            name=str(int(year)),  # Convertir l'année en entier pour l'affichage
            visible=(i == len(years) - 1),  # La dernière année est visible par défaut
        ))

fig.update_layout(
    title='Evolution of Top 10 Beers',
    xaxis=dict(title='Year', tickmode='array', tickvals=years, ticktext=[str(int(year)) for year in years]),  # Utiliser les années entières sur l'axe x
    yaxis=dict(title='Position in Top 10', autorange='reversed'),  # Inverser l'axe y
    showlegend=True,
    sliders=[{
        'active': len(years) - 1,
        'steps': [{'label': str(int(year)), 'method': 'update', 'args': [{'visible': [i == j for j in range(len(years))]}]} for i, year in enumerate(years)]
    }],
)

fig.add_traces(scatter_data)

fig.show()

In [44]:
import pandas as pd
import plotly.graph_objects as go

# ... (votre code pour créer le DataFrame et les classemen

fig = go.Figure()

# Ajouter un scatter pour chaque année entière
scatter_data = []
for i, year in enumerate(years):
    try:
        if float(year).is_integer():  # Exclure les années avec des décimales
            top_10 = df_filled[df_filled['year'] == int(year)].nlargest(10, 'new_rating')
            scatter_data.append(go.Scatter(
                x=[year] * 10,
                y=list(range(10, 0, -1)),  # Inverser l'axe y
                mode='markers+text',
                marker=dict(size=10),
                text=top_10['beer_name'],
                name=str(int(year)),  # Convertir l'année en entier pour l'affichage
                visible=(i == len(years) - 1),  # La dernière année est visible par défaut
                textposition='middle right',  # Ajuster la position du texte
            ))
    except ValueError:
        pass  # Ignore les valeurs non convertibles en float

fig.update_layout(
    title='Evolution of Top 10 Beers',
    xaxis=dict(title='Year', tickmode='array', tickvals=years, ticktext=[str(year) for year in years]),  # Utiliser les années entières sur l'axe x
    yaxis=dict(title='Position in Top 10', autorange='reversed'),  # Inverser l'axe y
    showlegend=False,  # Masquer la légende
    width=800,  # Ajuster la largeur de la figure
    sliders=[{
        'active': len(years) - 1,
        'steps': [{'label': str(year), 'method': 'update', 'args': [{'visible': [i == j for j in range(len(years))]}]} for i, year in enumerate(years)]
    }],
)

fig.add_traces(scatter_data)

fig.show()


In [101]:
import pandas as pd
import plotly.graph_objects as go

# ... (votre code pour créer le DataFrame et les classements)

fig = go.Figure()

# Ajouter un scatter pour chaque année entière
scatter_data = []
for i, year in enumerate(years):
    try:
        if float(year).is_integer():  # Exclure les années avec des décimales
            top_10 = df_filled[df_filled['year'] == int(year)].nlargest(10, 'new_rating')
            scatter_data.append(go.Scatter(
                x=[year] * 10,
                y=list(range(1,11)),  # Inverser l'axe y
                mode='markers+text',
                marker=dict(size=20),
                text=top_10['beer_name'],
                name=str(int(year)),  # Convertir l'année en entier pour l'affichage
                visible=(i == 0),  # Les deux dernières années sont visibles par défaut
                textposition='top right', 
                 textfont=dict(size=12) # Ajuster la position du texte
            ))
    except ValueError:
        pass  # Ignore les valeurs non convertibles en float

fig.update_layout(
    title='Evolution of Top 10 Beers',
    xaxis=dict(title='Year', tickmode='array', tickvals=years, ticktext=[str(year) for year in years], dtick=0.1),  # Utiliser les années entières sur l'axe x
    yaxis=dict(title='Position in Top 10', tickmode= 'linear', tickvals= list(range(1,11)), dtick=1,autorange='reversed'),  # Inverser l'axe y
    showlegend=False,
    height= 800,  # Masquer la légende
    width=1600,  # Ajuster la largeur de la figure
    sliders=[{
        'active': 0,
        'steps': [{'label': str(year), 'method': 'update', 'args': [{'visible': [i == j or i == j-1 or i == j+1 for j in range(len(years))]}]} for i, year in enumerate(years)]
    }],
)

fig.add_traces(scatter_data)

fig.show()


In [111]:

# ... (votre code pour créer le DataFrame et les classements)

fig = go.Figure()

# Ajouter un scatter pour chaque année entière
scatter_data = []
spacing = 0.1  # Ajustez l'espacement selon vos préférences

for i, year in enumerate(years):
    try:
        if float(year).is_integer():  # Exclure les années avec des décimales
            top_10 = df_filled[df_filled['year'] == int(year)].nlargest(10, 'new_rating')
            scatter_data.append(go.Scatter(
                x=[(i - len(years) / 2) * spacing] * 10,  # Centrer les années sur l'axe x
                y=list(range(1, 11)),
                mode='markers+text',
                marker=dict(size=15),
                text=top_10['beer_name'],
                name=str(int(year)),
                visible=(i == 0),  # La première année est visible par défaut
                textposition='top center',
                textfont=dict(size=12),
            ))
    except ValueError:
        pass  # Ignore les valeurs non convertibles en float

slider_steps = [{'label': str(year), 'method': 'update', 'args': [{'visible': [i == j or i == j + 1 or i == j - 1 for j in range(len(years))]}]} for i, year in enumerate(years)]

fig.update_layout(
    title='Evolution of Top 10 Beers',
    xaxis=dict(title='Year', tickmode='array', tickvals=[(i - len(years) / 2)*spacing for i in range(len(years))], ticktext=[str(year) for year in years], dtick=spacing),
    yaxis=dict(title='Position in Top 10', tickmode='linear', tickvals=list(range(1, 11)), dtick=1, autorange='reversed'),
    showlegend=False,
    height=800,
    width=1600,  # Ajuster la largeur de la figure
    sliders=[{
        'active': 0,  # La première année est active par défaut
        'steps': slider_steps,
    }],
)

fig.add_traces(scatter_data)

fig.show()


In [121]:
import pandas as pd
from raceplotly.plots import barplot

data = pd.read_csv('./../ada-2023-project-laremontada61/top_10_beers_by_year.csv')

my_raceplot = barplot(data,
                      item_column='beer_name',
                      value_column='new_rating',
                      time_column='year')

my_raceplot.plot(title = 'Top 10 beers in the world from 2000 to 2017',
                 item_label = 'Top 10 beers',
                 value_label = 'New rating',
                 frame_duration = 800)